# Information Visualization II
## School of Information, University of Michigan

## Week 1: 
- Multivariate/Multidimensional + Temporal

## Assignment Overview
### This assignment's objectives include:

- Review, reflect on, and apply different strategies for multidimensional/multivariate/temporal datasets

- Recreate visualizations and propose new and alternative visualizations using [Altair](https://altair-viz.github.io/) 

### The total score of this assignment will be 100 points consisting of:
- You will be producing four visualizations. Three of them will require you to follow the example closely, but the last will be fairly open-ended. For the last one, we'll also ask you to justify why you designed your visualization the way you did.

### Resources:
- Article by [FiveThirtyEight](https://fivethirtyeight.com) available  [online](https://fivethirtyeight.com/features/a-statistical-analysis-of-the-work-of-bob-ross/) (Hickey, 2014)
- The associated dataset on [Github](https://github.com/fivethirtyeight/data/tree/master/bob-ross)
- A dataset of all the [paintings from the show](https://github.com/jwilber/Bob_Ross_Paintings)
    
    
### Important notes:
1) Grading for this assignment is entirely done by manual inspection. For some of the visualizations, we'll expect you to get pretty close to our example (1-3). Problem 4 is more free-form.  

2) There are a few instances where our numbers do not align exactly with those from 538. We've pre-processed our data a little bit differently.

3) When turning in your PDF, please use the File -> Print -> Save as PDF option ***from your browser***. Do ***not*** use the File->Download as->PDF option. Complete instructions for this are under Resources in the Coursera page for this class.

In [220]:
# load up the resources we need
import zipfile as zip
import urllib.request
import os.path
from os import path
import pandas as pd
import altair as alt
import numpy as np
from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA
import ipywidgets as widgets
from IPython.display import display
from PIL import Image

In [221]:
pd.set_option('display.max_columns', None)

In [222]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Bob Ross

Today's assignment will have you working with artwork created by [Bob Ross](https://en.wikipedia.org/wiki/Bob_Ross). Bob was a very famous painter who had a televised painting show from 1983 to 1994. Over 13 seasons and approximately 400 paintings, Bob would walk the audience through a painting project. Often these were landscape images. Bob was famous for telling his audience to paint "happy trees" and sayings like, "We don't make mistakes, just happy little accidents." His soothing voice and bushy hair are well known to many generations of viewers.

If you've never seen an episode, I might suggest starting with [this one](https://www.youtube.com/watch?v=Fw6odlNp7_8). 

![bob ross](assets/bobrosspaints.png)

Bob Ross left a long legacy of art which makes for an interesting dataset to analyze. It's both temporally rich and has a lot of variables we can code. We'll be starting with the dataset created by 538 for their article on a [Statistical Analysis of Bob Ross](https://fivethirtyeight.com/features/a-statistical-analysis-of-the-work-of-bob-ross/). The authors of the article coded each painting to indicate what features the image contained (e.g., one tree, more than one tree, what kinds of clouds, etc.). 

In addition, we've downloaded a second dataset that contains the actual images. We know what kind of paint colors Bob used in each episode, and we have used that to create a dataset for you containing the color distributions. For example, we approximate how much '<font color='#614f4b'>burnt umber</font>' he used by measuring the distance (in color space) from each pixel in the image to the color. This is imperfect, of course (paints don't mix this way), but it'll be close enough for our analysis.

In [223]:
# the paints Bob used
rosspaints = ['alizarin crimson','bright red','burnt umber','cadmium yellow','dark sienna', 
              'indian yellow','indian red','liquid black','liquid clear','black gesso',
              'midnight black','phthalo blue','phthalo green','prussian blue','sap green',
              'titanium white','van dyke brown','yellow ochre']

# hex values for the paints above
rosspainthex = ['#94261f','#c06341','#614f4b','#f8ed57','#5c2f08','#e6ba25','#cd5c5c',
                '#000000','#ffffff','#000000','#36373c','#2a64ad','#215c2c','#325fa3',
                '#364e00','#f9f7eb','#2d1a0c','#b28426']

# boolean features about what an image includes
imgfeatures = ['Apple frame', 'Aurora borealis', 'Barn', 'Beach', 'Boat', 
               'Bridge', 'Building', 'Bushes', 'Cabin', 'Cactus', 
               'Circle frame', 'Cirrus clouds', 'Cliff', 'Clouds', 
               'Coniferous tree', 'Cumulis clouds', 'Decidious tree', 
               'Diane andre', 'Dock', 'Double oval frame', 'Farm', 
               'Fence', 'Fire', 'Florida frame', 'Flowers', 'Fog', 
               'Framed', 'Grass', 'Guest', 'Half circle frame', 
               'Half oval frame', 'Hills', 'Lake', 'Lakes', 'Lighthouse', 
               'Mill', 'Moon', 'At least one mountain', 'At least two mountains', 
               'Nighttime', 'Ocean', 'Oval frame', 'Palm trees', 'Path', 
               'Person', 'Portrait', 'Rectangle 3d frame', 'Rectangular frame', 
               'River or stream', 'Rocks', 'Seashell frame', 'Snow', 
               'Snow-covered mountain', 'Split frame', 'Steve ross', 
               'Man-made structure', 'Sun', 'Tomb frame', 'At least one tree', 
               'At least two trees', 'Triple frame', 'Waterfall', 'Waves', 
               'Windmill', 'Window frame', 'Winter setting', 'Wood framed']

# load the data frame
bobross = pd.read_csv("assets/bobross.csv")

# enable correct rendering (unnecessary in later versions of Altair)
alt.renderers.enable('default')

# uses intermediate json files to speed things up
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

We have a few variables defined for you that you might find useful for the rest of this exercise. First is the ```bobross``` dataframe which, has a row for every painting created by Bob (we've removed those created by guest artists).

In [224]:
# run to see what's inside
bobross.sample(5)

,EPISODE,TITLE,RELEASE_DATE,Apple frame,Aurora borealis,Barn,Beach,Boat,Bridge,Building,Bushes,Cabin,Cactus,Circle frame,Cirrus clouds,Cliff,Clouds,Coniferous tree,Cumulis clouds,Decidious tree,Diane andre,Dock,Double oval frame,Farm,Fence,Fire,Florida frame,Flowers,Fog,Framed,Grass,Guest,Half circle frame,Half oval frame,Hills,Lake,Lakes,Lighthouse,Mill,Moon,At least one mountain,At least two mountains,Nighttime,Ocean,Oval frame,Palm trees,Path,Person,Portrait,Rectangle 3d frame,Rectangular frame,River or stream,Rocks,Seashell frame,Snow,Snow-covered mountain,Split frame,Steve ross,Man-made structure,Sun,Tomb frame,At least one tree,At least two trees,Triple frame,Waterfall,Waves,Windmill,Window frame,Winter setting,Wood framed,painting_index,season,episode,num_colors,youtube_src,Black_Gesso,Bright_Red,Burnt_Umber,Cadmium_Yellow,Dark_Sienna,Indian_Red,Indian_Yellow,Liquid_Black,Liquid_Clear,Midnight_Black,Phthalo_Blue,Phthalo_Green,Prussian_Blue,Sap_Green,Titanium_White,Van_Dyke_Brown,Yellow_Ochre,Alizarin_Crimson,alizarin crimson,bright red,burnt umber,cadmium yellow,dark sienna,indian yellow,indian red,liquid black,liquid clear,black gesso,midnight black,phthalo blue,phthalo green,prussian blue,sap green,titanium white,van dyke brown,yellow ochre,img_url,week_number,year
21,S02E10,"""LAZY RIVER""",11/2/83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,278,2,10,12,https://www.youtube.com/embed/BW2wKKFvH1g,0,1,1,1,0,0,1,0,0,0,1,1,1,1,1,1,1,1,0.440230,0.450677,0.560978,0.370982,0.000000,0.333667,0.0,0.0,0.0,0.000000,0.000000,0.431295,0.446642,0.462576,0.371952,0.303800,0.397037,0.419378,https://raw.githubusercontent.com/jwilber/Bob_...,44,1983
130,S11E12,"""ROADSIDE BARN""",3/18/87,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,161,11,12,11,https://www.youtube.com/embed/vJpKhiXvXdA,0,1,0,1,1,0,1,0,0,1,0,0,1,1,1,1,1,1,0.491555,0.452883,0.000000,0.342467,0.497332,0.340680,0.0,0.0,0.0,0.000000,0.588858,0.000000,0.000000,0.435614,0.475242,0.237322,0.506903,0.448354,https://raw.githubusercontent.com/jwilber/Bob_...,12,1987
186,S16E04,"""MOUNTAIN MIRAGE""",9/7/88,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,87,16,4,12,https://www.youtube.com/embed/jfCsew_mz7A,0,1,0,1,1,0,1,0,0,1,1,0,1,1,1,1,1,1,0.390504,0.457842,0.000000,0.415270,0.325598,0.321799,0.0,0.0,0.0,0.000000,0.476101,0.529097,0.000000,0.565001,0.287313,0.370033,0.318284,0.388980,https://raw.githubusercontent.com/jwilber/Bob_...,36,1988
188,S16E07,"""DEEP WOODS""",9/28/88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,90,16,7,10,https://www.youtube.com/embed/rE5ZVs_YJfE,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,1,1,1,0.434441,0.503063,0.000000,0.471728,0.357766,0.000000,0.0,0.0,0.0,0.000000,0.450871,0.369208,0.000000,0.000000,0.307720,0.362868,0.330646,0.438721,https://raw.githubusercontent.com/jwilber/Bob_...,39,1988
296,S25E02,"""ENCHANTED FALLS OVAL""",9/1/92,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,322,25,2,6,https://www.youtube.com/embed/HklPkQ0lSKA,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0.593475,0.000000,0.000000,0.000000,0.000000,0.437772,0.0,0.0,0.0,0.424592,0.000000,0.000000,0.000000,0.000000,0.513545,0.200703,0.000000,0.554971,https://raw.githubusercontent.com/jwilber/Bob_...,36,1992


In the dataframe you will see an episode identifier (EPISODE, which contains the season and episode number), the image title (TITLE), the release date (RELEASE_DATE as well as another column for the year). There are also a number of boolean columns for the features coded by 538. A '1' means the feature is present, a '0' means it is not. A list of those columns is available in the ```imgfeatures``` variable.

In [225]:
# run to see what's inside
print(imgfeatures)

['Apple frame', 'Aurora borealis', 'Barn', 'Beach', 'Boat', 'Bridge', 'Building', 'Bushes', 'Cabin', 'Cactus', 'Circle frame', 'Cirrus clouds', 'Cliff', 'Clouds', 'Coniferous tree', 'Cumulis clouds', 'Decidious tree', 'Diane andre', 'Dock', 'Double oval frame', 'Farm', 'Fence', 'Fire', 'Florida frame', 'Flowers', 'Fog', 'Framed', 'Grass', 'Guest', 'Half circle frame', 'Half oval frame', 'Hills', 'Lake', 'Lakes', 'Lighthouse', 'Mill', 'Moon', 'At least one mountain', 'At least two mountains', 'Nighttime', 'Ocean', 'Oval frame', 'Palm trees', 'Path', 'Person', 'Portrait', 'Rectangle 3d frame', 'Rectangular frame', 'River or stream', 'Rocks', 'Seashell frame', 'Snow', 'Snow-covered mountain', 'Split frame', 'Steve ross', 'Man-made structure', 'Sun', 'Tomb frame', 'At least one tree', 'At least two trees', 'Triple frame', 'Waterfall', 'Waves', 'Windmill', 'Window frame', 'Winter setting', 'Wood framed']


The columns that contain the amount of each color in the paintings are listed in ```rosspaints```. There is also an analogous list variable called ```rosspainthex``` that has the hex values for the paints. These hex values are approximate.

In [226]:
# run to see what's inside
print("paint names",rosspaints)
print("")
print("hex values", rosspainthex)

paint names ['alizarin crimson', 'bright red', 'burnt umber', 'cadmium yellow', 'dark sienna', 'indian yellow', 'indian red', 'liquid black', 'liquid clear', 'black gesso', 'midnight black', 'phthalo blue', 'phthalo green', 'prussian blue', 'sap green', 'titanium white', 'van dyke brown', 'yellow ochre']

hex values ['#94261f', '#c06341', '#614f4b', '#f8ed57', '#5c2f08', '#e6ba25', '#cd5c5c', '#000000', '#ffffff', '#000000', '#36373c', '#2a64ad', '#215c2c', '#325fa3', '#364e00', '#f9f7eb', '#2d1a0c', '#b28426']


### Problem 1  (20 points)

As a warmup, we're going to have you recreate the [first chart from the Bob Ross article](assets/bob_ross_538.png) (source: [Statistical Analysis of Bob Ross](https://fivethirtyeight.com/features/a-statistical-analysis-of-the-work-of-bob-ross/)). This one simply shows a bar chart for the percent of images that have certain features. The Altair version is:

!["Bob Ross feature distribution"](assets/bob_ross_altair.png)

We'll be using the 538 theme for styling, so you don't have to do much beyond creating the chart (but do note that we want to see the percents, titles, and modifications to the axes).

You will replace the code for ```makeBobRossBar()``` and have it return an Altair chart.  We suggest you first create a table that contains the names of the features and the percents.  Something like this:

!["Sample Table](assets/feature_table.png)

Recall that this is the 'long form' representation of the data, which will make it easier to create a visualization with.

In [227]:
#It will be useful to know how many features are present so we can slice the dataframe accordingly
print(len(imgfeatures))

67


In [228]:
#This will give us the number of images, which we will use as the numerator to get the percentage (total # images/# times feature is present)
len(bobross['TITLE'])


381

In [229]:
def makeBobRossBar():
    # implement this function to return an altair chart
    # return alt.Chart(...)
    #Let's first create the table shown above
    
    bobross_copy = bobross
    
    bobross_copy = bobross_copy.drop(['EPISODE', 'TITLE', 'RELEASE_DATE'], axis=1)
    
    bobross_copy = bobross_copy.iloc[:, 0:67]
    
    bobross_copy = bobross_copy.T
    
    bobross_copy['Total'] = bobross_copy.sum(axis=1)
    
    bobross_copy['Percent'] = bobross_copy['Total']/381
    
    
    
    bobross_copy = bobross_copy.sort_values(by=['Percent'], ascending=False)
    
    bobross_copy = bobross_copy.round({'Percent': 2})
    
    bobross_copy = bobross_copy[(bobross_copy['Percent']>=0.02)]
    
    #bobross_copy['Percent'] = pd.Series(["{0:.0f}%".format(val * 100) for val in bobross_copy['Percent']], index = bobross_copy.index)
    
    bobross_copy = bobross_copy.loc[:,['Percent']]
    
    bobross_copy = bobross_copy.reset_index()
    
    
    bobross_copy = bobross_copy.rename(columns={'index': 'Feature'})
    
    names_l = bobross_copy['Feature'].to_list()

# make bar chart with mark_bar
    bars = alt.Chart(bobross_copy).mark_bar(size=20).encode(
    # encode x as the percent, and hide the axis
      x=alt.X(
        'Percent:Q',
        axis=None),
    y=alt.Y(
        # encode y using the name, use the movie name to label the axis, sort using the names_l
        'Feature:N',
         axis=alt.Axis(tickCount=5, title=''),
         # we give the sorting order to avoid alphabetical order
        sort = names_l
    )
)
    
    
    # we're going to overlay the text with the percentages, so let's make another visualization
# that's just text labels

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        # we'll use the percentage as the text
        text=alt.Text('Percent:Q',format='.0%'),
        
    )

    # finally, we're going to combine the bars and the text and do some styling
    bobross_chart = (text + bars).configure_mark(
    # we don't love the blue
    color='#29b6f6'
    ).configure_view(
    # we don't want a stroke around the bars
    strokeWidth=0
    ).configure_scale(
    # add some padding
    bandPaddingInner=0.2
    ).configure(
    background='#DCDCDC'
    ).properties(
    # set the dimensions of the visualization
    width=800,
    height=800
    ).properties(
    # add a title

     title={
      "text": ["The Paintings of Bob Ross"], 
      "subtitle": ["Percentage containing each element"],
      "subtitleColor": "black"
    }
)


    
    return  bobross_chart


  


In [230]:
# run this code to validate
alt.themes.enable('fivethirtyeight')
makeBobRossBar()

alt.LayerChart(...)

## Problem 2 (25 points)

The 538 article ([Statistical Analysis of Bob Ross](https://fivethirtyeight.com/features/a-statistical-analysis-of-the-work-of-bob-ross/)) has a long analysis of conditional probabilities. Essentially, we want to know the probability of one feature given another (e.g., what is the probability of Snow given Trees?). The article calculates this over the entire history of the show, but we would like to visualize these probabilities over time. Have they been constant? or evolving?  We will only be doing this for a few variables (otherwise, we'll have a matrix of over 3000 small charts). Specifically, we care about images that contain: 'At least one tree','At least two trees','Clouds','Grass','At least one mountain','Lake.' Each small multiple plot will be a line chart corresponding to the conditional probability over time. The matrix "cell" indicates which pairs of variables are being considered (e.g., probability of at least two trees given the probability of at least one tree is the 2nd row, first column in our example).

Your task will be to generate the small multiples plot below:

!["Small multiples"](assets/matrix_small.png)

The full image is [available here](assets/matrix_full.png). While your small multiples visualization should contain all this data, you can ***feel free to style it as you think is appropriate***. We will be grading (minimally) on aesthetics. Implement the code for the function: ```makeBobRossCondProb()``` to return this chart.

Some notes on doing this exercise:

* If you don't remember how to calculate conditional probabilities, take a look at the article. Remember, we want the conditional probabilities given the images in a specific year. This is simply an implementation of Bayes' Theorem. We implemented a function called ```condprobability(...)``` as you can see below. You can do the same or pick your own strategy for this.

* We suggest creating a long-form representation of the table for this data. For example, here's a sample of ours (you can use this to double this sample for your calculations):

!["Long form conditional probabilities](assets/cond_prob_table.png)

* There are a number of strategies to build the small-multiple plots. Some are easier than others. You will find in this case that some combinations of repeated charts and faceting will not work. However, you should be able to use the standard concatenation approaches in combination with repeated charts or faceting.

In [231]:
def condprobability(frame,column1,column2,year):
    # we suggest you implement this function to make your life easier. It should take a datafame as input,
    # the two columns we want the conditional probability for, and the year for which we want to compare
    # you can make variants of this function as you see fit
    
    df_filtered = frame[frame['year']==year]
 
    numerator = ((df_filtered [column1] == 1) & (df_filtered [column2] == 1)).sum()
    
    denominator = len(df_filtered [df_filtered [column2]==1])
    
    return numerator/denominator
    
    

In [232]:
import itertools
from altair import datum
def makeBobRossCondProb(totest=['At least one tree','At least two trees','Clouds','Grass','At least one mountain','Lake']):
    # implement this function to return an altair chart
    # note that we have created a default 'totest' variable that has the columns for which 
    # we want the pairwise analysis
    
    # return alt.Chart(...)
    
    bobross_copy = bobross

    
    bobross_copy = bobross_copy.loc[:,['At least one tree','At least two trees','Clouds','Grass','At least one mountain','Lake', 'year']]
    
    years = [1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,]

    totest_2 = ['At least one tree','At least two trees','Clouds','Grass','At least one mountain','Lake']
    
    
    combination_list = [totest, years, totest_2]    
    
    permutations = list(itertools.product(*combination_list))
    
    permutations_df = pd.DataFrame.from_records(permutations)
    
    permutations_df =  permutations_df.rename(columns={0: 'Key1', 1: 'year', 2: 'Key2' })
    
    #Let's create the conditional probability column using our permutations df
    permutations_df['prob'] = permutations_df.apply(lambda row: condprobability(bobross_copy, row.Key1, row.Key2, row.year), axis = 1)
    
    #Let's set up all the df's for filtering
    permutations_df_at_least_one_tree =  permutations_df[permutations_df['Key1']=='At least one tree']
    permutations_df_at_least_two_trees =  permutations_df[permutations_df['Key1']=='At least two trees']
    permutations_df_clouds =  permutations_df[permutations_df['Key1']=='Clouds']
    permutations_df_grass =  permutations_df[permutations_df['Key1']=='Grass']
    permutations_df_at_least_one_mountain = permutations_df[permutations_df['Key1']=='At least one mountain']
    permutations_df_lake = permutations_df[permutations_df['Key1']=='Lake']
    
    #Plot 1
    chart1 = alt.Chart(permutations_df_at_least_one_tree).mark_line().encode(
            x=alt.X('year:O',axis=alt.Axis(title=None, labelExpr="datum.value == 1983 || datum.value == 1987 || datum.value == 1991 ? datum.label : null")),
            y=alt.Y('prob:Q', axis=alt.Axis(title=['Probability of', 'At least one tree']), scale=alt.Scale(domain=(0,1))),
            facet=alt.Facet('Key2:N', title="Given...", sort=totest)
        ).properties(
            width=100,
            height=100
        )

    #Plot 2
    chart2 = alt.Chart(permutations_df_at_least_two_trees ).mark_line().encode(
            x=alt.X('year:O',axis=alt.Axis(title=None, labelExpr="datum.value == 1983 || datum.value == 1987 || datum.value == 1991 ? datum.label : null")),
            y=alt.Y('prob:Q', axis=alt.Axis(title=['Probability of', 'At least two trees']), scale=alt.Scale(domain=(0,1))),
            facet=alt.Facet('Key2:N', title="Given...", sort=totest)
        ).properties(
            width=100,
            height=100
        )
    
    
    #Plot 3
    chart3 = alt.Chart(permutations_df_clouds).mark_line().encode(
            x=alt.X('year:O',axis=alt.Axis(title=None, labelExpr="datum.value == 1983 || datum.value == 1987 || datum.value == 1991 ? datum.label : null")),
            y=alt.Y('prob:Q', axis=alt.Axis(title=['Probability of', 'Clouds']), scale=alt.Scale(domain=(0,1))),
            facet=alt.Facet('Key2:N', title="Given...", sort=totest)
        ).properties(
            width=100,
            height=100
        )
    
    
    
    #Plot 4
    chart4 = alt.Chart(permutations_df_grass).mark_line().encode(
            x=alt.X('year:O',axis=alt.Axis(title=None, labelExpr="datum.value == 1983 || datum.value == 1987 || datum.value == 1991 ? datum.label : null")),
            y=alt.Y('prob:Q', axis=alt.Axis(title=['Probability of', 'Grass']), scale=alt.Scale(domain=(0,1))),
            facet=alt.Facet('Key2:N', title="Given...", sort=totest)
        ).properties(
            width=100,
            height=100
        )  
    
    #Plot 5
        #Plot 4
    chart5 = alt.Chart(permutations_df_at_least_one_mountain).mark_line().encode(
            x=alt.X('year:O',axis=alt.Axis(title=None, labelExpr="datum.value == 1983 || datum.value == 1987 || datum.value == 1991 ? datum.label : null")),
            y=alt.Y('prob:Q', axis=alt.Axis(title=['Probability of', 'At least one mountain']), scale=alt.Scale(domain=(0,1))),
            facet=alt.Facet('Key2:N', title="Given...", sort=totest)
        ).properties(
            width=100,
            height=100
        )  
    
    
     #Plot 6
    chart6 = alt.Chart(permutations_df_lake).mark_line().encode(
            x=alt.X('year:O',axis=alt.Axis(title=None, labelExpr="datum.value == 1983 || datum.value == 1987 || datum.value == 1991 ? datum.label : null")),
            y=alt.Y('prob:Q', axis=alt.Axis(title=['Probability of', 'Lake']), scale=alt.Scale(domain=(0,1))),
            facet=alt.Facet('Key2:N', title="Given...", sort=totest)
        ).properties(
            width=100,
            height=100
        )  
    
    
    return alt.vconcat(chart1, chart2, chart3, chart4, chart5, chart6)

In [233]:
# run this cell to test your code
makeBobRossCondProb()

alt.VConcatChart(...)

### Additional comments

If you deviated from our example, please use this cell to give us additional information about your design choices and why you think they are an improvement.


## Problem 3 (25 points)

Recall that in some cases of multidimensional data a good strategy is to use dimensionality reduction to visualize the information. Here, we would like to understand how images are similar to each other in 'feature' space. Specifically, how similar are they based on the image features? Are images that have beaches close to those with waves? 

We are going to create a 2D MDS plot using the scikit learn package. We're going to do most of this for you in the next cell. Essentially we will use the euclidean distance between two images based on their image feature array to create the image. Your plot may look slightly different than ours based on the random seed (e.g., rotated or reflected), but in the end, it should be close. If you're interested in how this is calculated, we suggest taking a look at [this documentation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html)

Note that the next cell may take a minute or so to run, depending on the server.  

In [234]:
# create the seed
seed = np.random.RandomState(seed=3)

# generate the MDS configuration, we want 2 components, etc. You can tweak this if you want to see how
# the settings change the layout
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed, n_jobs=1)

# fit the data. At the end, 'pos' will hold the x,y coordinates
pos = mds.fit(bobross[imgfeatures]).embedding_

# we'll now load those values into the bobross data frame, giving us a new x column and y column
bobross['x'] = [x[0] for x in pos]
bobross['y'] = [x[1] for x in pos]

Your task is to implement the visualization for the MDS layout. We will be using a new mark, ```mark_image```, for this. You can read all about this mark on the Altair site [here](https://altair-viz.github.io/user_guide/marks.html#user-guide-image-mark). Note that we all already saved the images for you. They are accessible in the img_url column in the bobross table. You will use the ```url``` encode argument to mark_image to make this work.

In this case, we would also like to emphasize all the images that *have* a specific feature. So when you define your ```genMDSPlot()``` function below, it should take a key string as an argument (e.g., 'Beach') and visually highlight those images. A simple way to do this is to use a second mark underneath the image (e.g., a rectangle) that is a different color based on the absence or presence of the image.  Here's an example output for ```genMDSPlot("Palm trees")```:

!["mds"](assets/mds_small.png)

Click [here](assets/mds_large.png) for a large version of this image. Notice the orange boxes indicating where the Palm tree images are. Note that we have styled the MDS plot to not have axes. Recall that these are meaningless in MDS 'space' (this is not a scatterplot, it's a projection).

In [235]:
def genMDSPlot(key):
    # return an altair chart (e.g., return alt.Chart(...))
    # key is a string indicating which images should be visually highlighted (i.e., images containing the feature
    # should be made salient)
    
    bobross_copy = bobross[[key, 'x', 'y', 'img_url']]
    bobross_copy.columns = ['key', 'x', 'y', 'img_url']
    
    

    chart1 = alt.Chart(bobross_copy, width=800, height=800).mark_image(
        width=35,
        height=35
    ).encode(
        x=alt.X('x', axis=alt.Axis(title='', grid=False, tickCount=0)),
        y=alt.Y('y', axis=alt.Axis(title='', grid=False, tickCount=0)),
        url='img_url',
        color=alt.Color("key:N", legend=alt.Legend(title=key) )
        
    )
    
    
    chart2 = alt.Chart(bobross_copy, width=800, height=800).mark_rect(
        width=40,
        height=40
    ).encode(
        x=alt.X('x', axis=alt.Axis(title='', grid=False, tickCount=0)),
        y=alt.Y('y', axis=alt.Axis(title='', grid=False, tickCount=0)),
        color=alt.Color("key:N")
        
    )
    
    return alt.layer(chart2, chart1)

We are going to create an interactive widget that allows you to select the feature you want to be highlighted. If you implemented your ```genMDSPlot``` code correctly, the plot should change when you select new items from the list. We would ordinarily do this directly in Altair, but because we don't have control over the way you created your visualization, it's easiest for us to use the widgets built into Jupyter.

It should look something like this:

!["mds interactive"](assets/interactive_mds.png)

In [236]:
# note that it might take a few seconds for the images to download
# depending on your internet connection

output = widgets.Output()

def clicked(b):
    output.clear_output()
    with output:
        highlight = filterdrop.value
        if (highlight == ""):
            print("please enter a query")
        else:
            genMDSPlot(highlight).display()


featurecount = bobross[imgfeatures].sum()

filterdrop = widgets.Dropdown(
    options=list(featurecount[featurecount > 2].keys()),
    description='Highlight:',
    disabled=False,
)

filterdrop.observe(clicked)

display(filterdrop,output)

with output:
    genMDSPlot('Barn').display()


Dropdown(description='Highlight:', options=('Barn', 'Beach', 'Bridge', 'Bushes', 'Cabin', 'Cactus', 'Cirrus cl…

Output()

## Problem 4 (30 points: 25 for solution, 5 for explanation)

Your last problem is fairly open-ended in terms of visualization. We would like to analyze the colors used in different images for a given season as a small multiples plot. You can pick how you represent your small multiples, but we will ask you to defend your choices below.  You must implement the function ```colorSmallMultiples(season)``` that takes a season number as input (e.g., 2) and returns an Altair chart. 

You can go something as simple as this:

!["simple small multiples"](assets/bob_ross_color_glyph.png)

This visualization has a row for every painting and a colored circle (in the color of the paint). The circle is sized based on the amount of the corresponding paint that is used in the image. 

You can also go to something as crazy as this:

!["face small multiples"](assets/bob_ross_face.png)

Here, we've overlaid circles as curls in Bob's massive hair. We're not claiming this is an effective solution, but you're welcome to do this (or anything else) as long as you describe the pros and cons of your choices. And, yes, we generated both examples using Altair.

Again, the relevant columns are available are listed in ```rosspaints``` (there are 18 of them). The values range from 0 to 1 based on the fraction of pixel color allocated to that specific paint.  The ```rosspainthex``` has the corresponding hex values for the paint color. 


In [237]:
def colorSmallMultiples(season):
    
    # return an Altair chart
    # season is the integer representing the season of the show are interested in. Limit your images
    # to that season in the small multiples display.
    
#     paint names ['alizarin crimson', 'bright red', 'burnt umber', 'cadmium yellow', 'dark sienna', 'indian yellow', 'indian red', 'liquid black', 'liquid clear', 'black gesso', 'midnight black', 'phthalo blue', 'phthalo green', 'prussian blue', 'sap green', 'titanium white', 'van dyke brown', 'yellow ochre']

#     hex values ['#94261f', '#c06341', '#614f4b', '#f8ed57', '#5c2f08', '#e6ba25', '#cd5c5c', '#000000', '#ffffff', '#000000', '#36373c', '#2a64ad', '#215c2c', '#325fa3', '#364e00', '#f9f7eb', '#2d1a0c', '#b28426']



    
    
    bobross_copy = bobross
    
    bobross_copy =  bobross_copy[bobross_copy['season']== season]
    
    plot1 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color = '#94261f').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['alizarin crimson'])),
    size=alt.Size('alizarin crimson:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot2 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#c06341').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['bright red'])),
    size=alt.Size('bright red:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot3 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#614f4b').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['burnt umber'])),
    size=alt.Size('burnt umber:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    plot4 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#f8ed57').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['cadmium yellow'])),
    size=alt.Size('cadmium yellow:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    plot5 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#5c2f08').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['dark sienna'])),
    size=alt.Size('dark sienna:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot6 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#e6ba25').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['indian yellow'])),
    size=alt.Size('indian yellow:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot7 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#cd5c5c').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['indian red'])),
    size=alt.Size('indian red:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    plot8 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#000000').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['liquid black'])),
    size=alt.Size('liquid black:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot9 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#ffffff').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['liquid clear'])),
    size=alt.Size('liquid clear:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    plot10 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#000000').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['black gesso'])),
    size=alt.Size('black gesso:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot11 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#36373c').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['midnight black'])),
    size=alt.Size('midnight black:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot12 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#2a64ad').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['phthalo blue'])),
    size=alt.Size('phthalo blue:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    plot13 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#215c2c').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['phthalo green'])),
    size=alt.Size('phthalo green:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    plot14 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#325fa3').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['prussian blue'])),
    size=alt.Size('prussian blue:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    
    plot15 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#364e00').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['sap green'])),
    size=alt.Size('sap green:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    
    plot16 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#f9f7eb').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['titanium white'])),
    size=alt.Size('titanium white:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    
    plot17 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#2d1a0c').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['van dyke brown'])),
    size=alt.Size('van dyke brown:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    
    plot18 = alt.Chart(bobross_copy, width=50, height=400).mark_circle(color='#b28426').encode(
    y=alt.Y('TITLE', axis=alt.Axis(title=['yellow ochre'])),
    size=alt.Size('yellow ochre:Q', scale=alt.Scale(domain=(0,1)), legend=None))
    
    
    
    
    bobross_chart_1 = alt.hconcat(plot1, plot2, plot3, plot4, plot5, plot6)
    
    
    bobross_chart_2 = alt.hconcat(plot7, plot8, plot9, plot10, plot11, plot12)
    
    bobross_chart_3 = alt.hconcat(plot13, plot14, plot15, plot16, plot17, plot18)
    
    
    bobross_chart_final = alt.vconcat(bobross_chart_1, bobross_chart_2, bobross_chart_3)
    
    return bobross_chart_final
    
    

    
    
   


In [238]:
# run this to test your code for season 1
colorSmallMultiples(1)

alt.VConcatChart(...)

In [239]:
# run this to test your code for season 2
colorSmallMultiples(2)

alt.VConcatChart(...)

### Explain your choices

Explain your design here. Describe the pros and cons in terms of visualization principles.


As I was deciding upon what mark encoding to use for this problem, I was torn between utilizing a stacked bar chart through mark_bar(), segmenting each painting into its own bar, and divvying up that bar by color based on the percentage of that color utilized in each respective painting, while also encoding each chunk as its respective color, and the more visually simplistic circle charts shown above. Ultimately, I decided to run with the latter approach, primarily because I wanted to choose a visualization technique that prioritized interpretability, minimized the data-ink ratio, and minimized excess chart junk. I would argue that this approach is acceptably expressive, as all of the necessary data - the amount of each color in each painting for that season, encoded by the size of the circles - is present, and all unnecessary data is absent. In regards to effectiveness, I believe that would depend on the particular aims of the viewer. If the viewer was interested in seeing the percentage of each color utilized for each painting, there wouldn't have been a very intuitive way of accomplishing that with this approach. Furthermore, if the viewer was interested in doing cross-color comparison - how much of one color is present in one painting vs another painting - it is almost unfeasible to do so with this approach. This is where a stacked bar chart would have been more advantageous, as the viewer would have been able to draw their eye across each bar with ease to compare the amount of color used within each painting. Where this approach is most effective, I believe, is for a high-level single color analysis(how much an individual color is utilized across an entire season's paintings), as well as for the detection of extreme outliers(individual paintings containing 0.0% of a certain color, or an individual painting having a drastically larger or smaller amount of color used as compared to others).